In [72]:
import json
import glob
import pandas as pd
from pandas import json_normalize

#from sklearn.metrics import f1_score
#from sklearn.metrics import roc_auc_score #average_precision_score
from sklearn.metrics import precision_recall_fscore_support

In [206]:
ncRNA = "translate_s100/Homo_sapiens_ncRNA_03_dayhoff_k13_refsprot_t1e9_jacc0.5.readsummary.tsv"
qfo = "translate_s100/Homo_sapiens_qfo_dna_03_dayhoff_k13_refsprot_t1e9_jacc0.5.readsummary.tsv"
nc = pd.read_csv(ncRNA, sep="\t",  index_col=["read_id"])
cd = pd.read_csv(qfo, sep="\t", index_col=["read_id"])
testDF = pd.concat([nc,cd])

In [207]:
testDF.shape

(200, 5)

In [208]:
testDF.reset_index(inplace=True)
testDF.head()

,read_id,frames_coding,frames_stop,frames_noncoding,classification,filename
0,read31734308/ENST00000609363.1 havana:known ch...,2,0,4,coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...
1,read11626489/ENST00000424955.1 havana:known ch...,0,6,0,non-coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...
2,read33406805/ENST00000398275.4 ensembl_havana_...,1,2,3,coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...
3,read1331888/ENST00000362764.1 ncrna:known chro...,0,6,0,non-coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...
4,read34338972/ENST00000415573.1 havana:known ch...,0,5,1,non-coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...


In [242]:
testDF.loc[1]["read_id"]

'read11626489/ENST00000424955.1 havana:known chromosome:GRCh37:1:103817769:103828355:1 gene:ENSG00000232753.1 gene_biotype:lincRNA transcript_biotype:lincRNA;mate1:131-280;mate2:269-418'

In [218]:
#def find_origin_csv(row):
#    if "qfo" in row["filename"][0]:
#        row["origin"] = "coding"
#    elif "ncRNA" in row["filename"][0]:
#        row["origin"] = "non-coding"
#    return row
#testDF.apply(find_origin_csv, axis=1)

testDF["origin"] = ["non-coding"]*100 + ["coding"]*100

In [219]:
testDF.head()

,read_id,frames_coding,frames_stop,frames_noncoding,classification,filename,origin
0,read31734308/ENST00000609363.1 havana:known ch...,2,0,4,coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...,non-coding
1,read11626489/ENST00000424955.1 havana:known ch...,0,6,0,non-coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...,non-coding
2,read33406805/ENST00000398275.4 ensembl_havana_...,1,2,3,coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...,non-coding
3,read1331888/ENST00000362764.1 ncrna:known chro...,0,6,0,non-coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...,non-coding
4,read34338972/ENST00000415573.1 havana:known ch...,0,5,1,non-coding,/home/ntpierce/2020-simulate-rnaseq/QfO_verteb...,non-coding


In [220]:
testDF["classification"].value_counts()

coding        104
non-coding     96
Name: classification, dtype: int64

In [221]:
testDF["origin"].value_counts()

coding        100
non-coding    100
Name: origin, dtype: int64

In [222]:
# precision_recall_fscore_support returns:
#    precisionfloat (if average is not None) or array of float, shape = [n_unique_labels]
#    recallfloat (if average is not None) or array of float, , shape = [n_unique_labels]
#    fbeta_scorefloat (if average is not None) or array of float, shape = [n_unique_labels]
#    supportNone (if average is not None) or array of int, shape = [n_unique_labels]
#The number of occurrences of each label in y_true.

# precision = TP/ (TP + FP)
# recall = TP/ (TP + FN)

precision_recall_fscore_support(testDF["origin"].tolist(), testDF["classification"].tolist()) #labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

(array([0.90384615, 0.9375    ]),
 array([0.94, 0.9 ]),
 array([0.92156863, 0.91836735]),
 array([100, 100]))

In [223]:
precision_recall_fscore_support(testDF["origin"].tolist(), testDF["classification"].tolist(), labels=['coding', 'non-coding']) #labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

(array([0.90384615, 0.9375    ]),
 array([0.94, 0.9 ]),
 array([0.92156863, 0.91836735]),
 array([100, 100]))

In [224]:
# average = "micro"
# Calculate metrics globally by counting the total true positives, false negatives and false positives.

precision_recall_fscore_support(testDF["origin"].tolist(), testDF["classification"].tolist(), labels=['coding', 'non-coding'], average="micro") #labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
# returns global precision, global recall, global fbeta

(0.92, 0.92, 0.92, None)

In [225]:
from sklearn.metrics import classification_report

In [226]:
 print(classification_report(testDF["origin"].tolist(), testDF["classification"].tolist(), labels=['coding', 'non-coding']))#, average="micro") #labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')


              precision    recall  f1-score   support

      coding       0.90      0.94      0.92       100
  non-coding       0.94      0.90      0.92       100

    accuracy                           0.92       200
   macro avg       0.92      0.92      0.92       200
weighted avg       0.92      0.92      0.92       200



In [227]:
# lets read in json summary and csv for this exact file and double check my summary script
json_coding = "translate_Hsapiens/Homo_sapiens_qfo_dna_03_dayhoff_k13_refsprot_t1e9.json"
json_nc = "translate_Hsapiens/Homo_sapiens_ncRNA_03_dayhoff_k13_refsprot_t1e9.json"
#sencha_csv = "translate_Hsapiens/Homo_sapiens_ncRNA_qfo_dna_03_dayhoff_k13_refsprot_t1e9.csv"
filename_list = [json_coding, json_nc]

In [228]:
def build_summary_df(filename_list):
    all_info=[]
    for fi in filename_list:
        with open(fi) as f:
            all_info.append(json_normalize(json.load(f)))
    return all_info

In [229]:
jsonInfo = build_summary_df(filename_list)
jDF = pd.concat(jsonInfo).fillna(0)

In [230]:
jDF["total_reads"] = jDF.groupby((jDF.columns.str.split(".").str[0].str.contains("categorization_counts")),axis=1).sum()[True]

In [231]:
jDF["categorization_counts.Coding"]


0    94
0    10
Name: categorization_counts.Coding, dtype: int64

In [250]:
def find_origin(row):
    if "qfo" in row["input_files"][0]:
        row["origin"] = "coding"
        row["true_positives"] = row["categorization_counts.Coding"]
        row["true_negatives"] = 0
        row["false_positives"] = 0 
        row["false_negatives"] = row["total_reads"] - row["categorization_counts.Coding"]
    elif "ncRNA" in row["input_files"][0]:
        row["origin"] = "noncoding"
        row["true_positives"] = 0
        row["true_negatives"] = row["total_reads"] - row["categorization_counts.Coding"]
        row["false_positives"] = row["categorization_counts.Coding"]
        row["false_negatives"]= 0
    return row

jDF = jDF.apply(find_origin, axis=1)

In [251]:
jDF["true_negatives"]

0     0
0    90
Name: true_negatives, dtype: int64

In [252]:
metricDF = jDF.groupby(["peptide_alphabet", "peptide_ksize", "peptide_bloom_filter"]).sum()[["true_positives", "true_negatives", "false_positives", "false_negatives"]]

In [253]:
metricDF

,,,true_positives,true_negatives,false_positives,false_negatives
peptide_alphabet,peptide_ksize,peptide_bloom_filter,,,,
dayhoff,13,output_simreads/sencha/index/refsprot_t1e9_dayhoff_k13.index,94,90,10,6


In [254]:
metricDF["precision"] = metricDF["true_positives"]/(metricDF["true_positives"] + metricDF["false_positives"])
metricDF["recall"] = metricDF["true_positives"]/(metricDF["true_positives"] + metricDF["false_negatives"])
metricDF["F1_score"] = 2 * (metricDF["precision"] * metricDF["recall"]) / (metricDF["precision"] + metricDF["recall"])

In [255]:
metricDF

,,,true_positives,true_negatives,false_positives,false_negatives,precision,recall,F1_score
peptide_alphabet,peptide_ksize,peptide_bloom_filter,,,,,,,
dayhoff,13,output_simreads/sencha/index/refsprot_t1e9_dayhoff_k13.index,94,90,10,6,0.903846,0.94,0.921569
